In [2]:
# imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import fbprophet

In [3]:
# online resources
# 1. OpenMeta: FbProphet Documentation: https://facebook.github.io/prophet/
# 2. TS forecasting with fbprophet: https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/
# 3. Parameter tuning for fbprophet: https://towardsdatascience.com/implementing-facebook-prophet-efficiently-c241305405a3

In [42]:
def generate_time_series(start_date_str, end_date_str, interval_min):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d %H:%M")
    interval = timedelta(hours=interval_min/60)
    output = []
    while start_date <= end_date:
        output.append(start_date.strftime("%Y-%m-%d %H:%M:%S"))
        start_date += interval
    return output

In [43]:
# navigate to price files
os.chdir("C:\\Users\\joche\\FIM Kernkompetenzzentrum\\Projekt VIdES - Dokumente\\General\\07_Arbeitsordner\\02_Daten_und_Simulationsvorbereitung\\Strompreise\\historische strompreise")

In [44]:
# read in and display historic price data

# create dummy series and df to append to
series = pd.Series(dtype="float64")
df = pd.read_csv(os.listdir()[0])
for i in os.listdir():
    # extract column name from file name
    s = "price_"
    s += i.split(".",2)[0][-4:]
    # read in file
    tmp = pd.read_csv(i, header=None)
    # assign to series
    series = pd.concat([series, tmp.iloc[:,0]])
    # assign to df
    tmp.set_axis([s], axis=1, inplace=True)
    df[s] = tmp

# reset series index, transform to df, and rename column
price_series = pd.DataFrame(series.reset_index()[0])
price_series.columns = ["price"]
#df_price_series = df_price_series.columns = ["price"]
# insert two empty rows and interpolate missing values
price_series.loc[price_series.shape[0]] = [np.nan]
price_series.loc[price_series.shape[0]] = [np.nan]
price_series.interpolate(inplace=True)
# generate hourly time series from 2016 to 2020
time_series = generate_time_series(start_date_str="2016-01-01 00:00", end_date_str="2020-12-31 23:00", interval_min=60)
# transform time series to df
df_time_series = pd.DataFrame(time_series, index=time_series)
# assign index of df_time_series to price_series
price_series.index = df_time_series.index

In [45]:
# navigate to pv and wind load profile file
os.chdir('C:\\Users\\joche\\FIM Kernkompetenzzentrum\\Projekt VIdES - Dokumente\\General\\07_Arbeitsordner\\02_Daten_und_Simulationsvorbereitung\\Strompreise')
# read in file
file2 = "pvLoad_windLoad_electricity_timeseries.xlsx"
pv_series = pd.read_excel(file2, sheet_name="pv")
wind_series = pd.read_excel(file2, sheet_name="wind")
pv_2030 = pd.read_excel(file2, sheet_name="pv_2030")
wind_2030 = pd.read_excel(file2, sheet_name="wind_2030")
# pv_series and wind_series: Convert date to datetime, set as index and remove cap_factor
pv_series['datetime'] = pd.to_datetime(pv_series['datetime'])
wind_series['datetime'] = pd.to_datetime(wind_series['datetime'])
pv_series.set_index('datetime', inplace=True)
wind_series.set_index('datetime', inplace=True)
pv_series.drop('pv_cap_factor', axis=1, inplace=True)
wind_series.drop('wind_cap_factor', axis=1, inplace=True)
price_series.index = wind_series.index
# pv_2030 and wind_2030: Convert date to datetime and set as index
pv_2030['datetime'] = pd.to_datetime(pv_2030['datetime'])
wind_2030['datetime'] = pd.to_datetime(wind_2030['datetime'])
pv_2030.set_index('datetime', inplace=True)
wind_2030.set_index('datetime', inplace=True)

In [46]:
# prepare dataframe df for model
df = price_series
df['ds'] = price_series.index
df['y'] = df['price']
df['pv_mwh'], df['wind_mwh'] = pv_series, wind_series
df.reset_index(inplace=True)
df.drop('datetime', axis=1, inplace=True)
df.drop('price', axis=1, inplace=True)
print('df:\n', df)
# prepare dataframe df_pred for predictions
df_pred = pv_2030
df_pred['ds'] = pv_2030.index
df_pred['pv_mwh'], df_pred['wind_mwh'] = pv_2030.pv_load_mwh, wind_2030.wind_load_mwh
df_pred.reset_index(inplace=True)
df_pred.drop('datetime', axis=1, inplace=True)
df_pred.drop('pv_load_mwh', axis=1, inplace=True)
print('df_pred:\n', df_pred)

df:
                        ds      y  pv_mwh   wind_mwh
0     2016-01-01 00:00:00  22.39     0.0  3898.8663
1     2016-01-01 01:00:00  20.59     0.0  3690.5645
2     2016-01-01 02:00:00  16.81     0.0  3364.5269
3     2016-01-01 03:00:00  17.41     0.0  3183.3949
4     2016-01-01 04:00:00  17.02     0.0  3056.6025
...                   ...    ...     ...        ...
43843 2020-12-31 19:00:00  52.44     0.0  8179.3260
43844 2020-12-31 20:00:00  51.86     0.0  6835.1520
43845 2020-12-31 21:00:00  52.26     0.0  5910.0120
43846 2020-12-31 22:00:00  52.26     0.0  5159.0160
43847 2020-12-31 23:00:00  52.26     0.0  4544.0700

[43848 rows x 4 columns]
df_pred:
                       ds  pv_mwh  wind_mwh
0    2030-01-01 00:00:00     0.0   77400.0
1    2030-01-01 01:00:00     0.0   86500.0
2    2030-01-01 02:00:00     0.0   96160.0
3    2030-01-01 03:00:00     0.0  101460.0
4    2030-01-01 04:00:00     0.0  111560.0
...                  ...     ...       ...
8755 2030-12-31 19:00:00     0.0  

In [47]:
# function to evaluate model
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
def prediction_diagnostics(model, df, start_day, end_day):
    # model overview: seasonality components
    df_tmp = df[(df['ds'] >= start_day) & (df['ds'] < end_day)]
    forecast_tmp = model.predict(df_tmp[['ds', 'pv_mwh', 'wind_mwh']])
    model.plot_components(forecast_tmp)
    # prediction overview: line plot
    eval_tmp = forecast_tmp[['ds', 'yhat']]
    # consider 'y' only for in-sample predictions
    if 'y' in df_tmp.columns:
        eval_tmp['y'] = df_tmp.y.values
    eval_tmp.index = eval_tmp.ds
    eval_tmp.drop('ds', axis=1, inplace=True)
    eval_tmp.plot(figsize=(10,8))
    # details: error metrics for in-sample predictions
    if 'y' in df_tmp.columns:
        print('MSE:\t', mean_squared_error(eval_tmp.y, eval_tmp.yhat))
        print('MAPE:\t', mean_absolute_percentage_error(eval_tmp.y, eval_tmp.yhat))

In [48]:
# model 1: Baseline
m1 = Prophet()
m1.add_regressor('pv_mwh')
m1.add_regressor('wind_mwh')
m1.fit(df)

C:\Users\joche\anaconda3\envs\fim_ml\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [31]:
# model 2: Increasing seasonality
m2 = Prophet(seasonality_mode='multiplicative')
m2.add_regressor('pv_mwh')
m2.add_regressor('wind_mwh')
m2.fit(df)

C:\Users\joche\anaconda3\envs\fim_ml\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [32]:
# model 3: Increasing seasonality and increasing mean
m3 = Prophet(seasonality_mode='multiplicative', growth='linear')
m3.add_regressor('pv_mwh')
m3.add_regressor('wind_mwh')
m3.fit(df)

C:\Users\joche\anaconda3\envs\fim_ml\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [49]:
# suppress fbprophet's warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [10]:
# in-sample evaluation: m1
%matplotlib qt
prediction_diagnostics(m1, df, start_day='2017-01-01', end_day='2018-01-01')

MSE:	 119.87816440941265
MAPE:	 25048387293950.12


In [47]:
# in-sample evaluation: m2
%matplotlib qt
prediction_diagnostics(m2, df, start_day='2019-01-01', end_day='2020-01-01')

MSE:	 93.6670100983144
MAPE:	 11870425823498.248


In [46]:
# in-sample evaluation: m3
%matplotlib qt
prediction_diagnostics(m3, df, start_day='2019-01-01', end_day='2020-01-01')

MSE:	 93.6670100983144
MAPE:	 11870425823498.248


In [50]:
# out-of-sample evaluation: m1
%matplotlib qt
prediction_diagnostics(m1, df_pred, start_day='2030-01-01', end_day='2030-12-31')

In [51]:
df_tmp = df_pred[(df_pred['ds'] >= '2030-01-01')]
df_tmp

,ds,pv_mwh,wind_mwh
0,2030-01-01 00:00:00,0.0,77400.0
1,2030-01-01 01:00:00,0.0,86500.0
2,2030-01-01 02:00:00,0.0,96160.0
3,2030-01-01 03:00:00,0.0,101460.0
4,2030-01-01 04:00:00,0.0,111560.0
...,...,...,...
8755,2030-12-31 19:00:00,0.0,30060.0
8756,2030-12-31 20:00:00,0.0,25120.0
8757,2030-12-31 21:00:00,0.0,21720.0
8758,2030-12-31 22:00:00,0.0,18960.0


In [53]:
# read in price data 2016-2020: df_prices
# navigate to price files
os.chdir("C:\\Users\\joche\\FIM Kernkompetenzzentrum\\Projekt VIdES - Dokumente\\General\\07_Arbeitsordner\\02_Daten_und_Simulationsvorbereitung\\Strompreise\\historische strompreise")
# create dummy series and df to append to
series = pd.Series(dtype="float64")
df = pd.read_csv(os.listdir()[0])
for i in os.listdir():
    # extract column name from file name
    s = "price_"
    s += i.split(".",2)[0][-4:]
    # read in file
    tmp = pd.read_csv(i, header=None)
    # assign to series
    series = pd.concat([series, tmp.iloc[:,0]])
    # assign to df
    tmp.set_axis([s], axis=1, inplace=True)
    df[s] = tmp
# disregard first column of df since it's a duplicate
df_prices = df.iloc[:,1:]
# truncate at index 8760
df_prices = df_prices.truncate(after=len(df_pred))
# interpolate missing values (at the end)
df_prices.interpolate()
df_prices

# extract df_price2030
# df_tmp = df_pred[(df_pred['ds'] >= '2030-01-01') & (df_pred['ds'] < '2030-12-31')]
df_tmp = df_pred[(df_pred['ds'] >= '2030-01-01')]
forecast_tmp = m1.predict(df_tmp[['ds', 'pv_mwh', 'wind_mwh']])
df_price2030 = forecast_tmp[['ds', 'yhat']]
# append price_2030
df_prices['price_2030'] = df_price2030[['yhat']]
df_prices = df_prices.truncate(after=8759)
df_prices

,price_2016,price_2017,price_2018,price_2019,price_2020,price_2030
0,22.39,20.96,-5.27,28.32,36.550,44.214693
1,20.59,20.90,-29.99,10.07,32.320,35.680180
2,16.81,18.13,-56.65,-4.08,30.850,27.554146
3,17.41,16.03,-63.14,-9.91,30.140,24.436901
4,17.02,16.43,-64.62,-7.41,30.170,19.739621
...,...,...,...,...,...,...
8755,43.32,7.92,58.28,46.00,46.880,101.499830
8756,41.03,4.06,50.01,42.20,43.220,101.249814
8757,40.55,5.30,45.06,39.74,37.430,99.561101
8758,33.06,1.86,48.93,38.88,38.540,98.245486


In [20]:
import pandas_profiling
os.chdir('..')
# summary statistics of df_price2030
df_prices_report = pandas_profiling.ProfileReport(df_prices)
df_prices_report.to_file("df_prices_report.html")
display(df_prices.describe().transpose())

INFO:visions.backends:Pandas backend loaded 1.4.2
INFO:visions.backends:Numpy backend loaded 1.22.3
INFO:visions.backends:Pyspark backend NOT loaded
INFO:visions.backends:Python backend loaded


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

,count,mean,std,min,25%,50%,75%,max
price_2016,8761.0,28.964373,12.484463,-130.090000,22.310000,28.230000,34.950000,104.960000
price_2017,8760.0,34.188527,17.659501,-83.060000,27.780000,33.825000,40.570000,163.520000
price_2018,8760.0,44.468920,17.771203,-76.010000,34.455000,45.090000,54.870000,128.260000
price_2019,8760.0,37.666600,15.517500,-90.010000,31.060000,38.060000,46.270000,121.460000
price_2020,8761.0,30.432294,17.489462,-83.940000,21.710000,30.960000,40.200000,200.040000
price_2030,8736.0,73.138729,29.077654,-37.943509,60.137033,78.272708,93.499606,127.017825


In [54]:
# rename an reshape df_price2030
df_price2030.index = df_price2030.ds
df_price2030.drop('ds', axis=1, inplace=True)
df_price2030.rename(columns = {'yhat' : 'price_2030'}, inplace=True)
df_price2030

,price_2030
ds,
2030-01-01 00:00:00,44.214693
2030-01-01 01:00:00,35.680180
2030-01-01 02:00:00,27.554146
2030-01-01 03:00:00,24.436901
2030-01-01 04:00:00,19.739621
...,...
2030-12-31 19:00:00,101.499830
2030-12-31 20:00:00,101.249814
2030-12-31 21:00:00,99.561101


In [55]:
# define function to transform hourly to quarter hourly data
def hourly_to_quarter_hourly_df(df_h):
    # for each row, add three empty ones
    empty_rows = 4
    df_h.index = range(0, empty_rows*len(df_h), empty_rows)
    df_h = df_h.reindex(index=range(empty_rows*len(df_h)))

    # generate quarter_hourly interval series from df_h's start date on
    r = str(df_h.ds[0]).split(" ", 1)[0]
    day, month, year = int(r.split("-")[2]), int(r.split("-")[1]), int(r.split("-")[0])
    start_date = datetime(year,month,day)
    quarter_hourly = [start_date + timedelta(minutes=15*x) for x in range(0,len(df_h))]
    quarter_hourly_list = [x.strftime('%Y-%m-%d %H:%M') for x in quarter_hourly]
    # generate dataframe
    quarter_hourly_list = pd.DataFrame(quarter_hourly_list, columns=['ds'])

    # insert quarter-hourly list into df_h -> df_qh
    df_h['ds'] = quarter_hourly_list['ds']
    df_qh = df_h[['ds', 'yhat']]

    # interpolate hourly to quarter-hourly values: linear interpolation
    df_qh_output = df_qh.interpolate(method='linear')
    df_qh_output = df_qh_output.assign(price_qh = lambda x: (x['yhat'] / 1)) # -> division by four only with capacity factors, not with prices

    return df_qh_output

In [56]:
# prepare df_price2030_qh for export
df_price2030_qh = hourly_to_quarter_hourly_df(forecast_tmp[['ds', 'yhat']])
df_price2030_qh.drop('yhat', axis=1, inplace=True)
df_price2030_qh.set_index('ds', inplace=True)
df_price2030_qh.rename(columns={'price_qh' : 'price_EUR_MWh'}, inplace=True)
# from EUR/MWh to ct/kWh
df_price2030_qh = df_price2030_qh.assign(price_ct_kWh = lambda x: (x['price_EUR_MWh'] / 10))
df_price2030_qh

,price_EUR_MWh,price_ct_kWh
ds,,
2030-01-01 00:00,44.214693,4.421469
2030-01-01 00:15,42.081065,4.208107
2030-01-01 00:30,39.947437,3.994744
2030-01-01 00:45,37.813808,3.781381
2030-01-01 01:00,35.680180,3.568018
...,...,...
2030-12-31 22:45,97.703712,9.770371
2030-12-31 23:00,97.523121,9.752312
2030-12-31 23:15,97.523121,9.752312


In [57]:
# write df_price2030 to .xlsx
os.chdir("C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\02_Daten_und_Simulationsvorbereitung\Strompreise")
df_price2030_qh.to_excel('prices_2030_v3.xlsx')